# 羊驼包 - 简历筛选器 📄

<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/llama_hub/llama_pack_resume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>

这个示例向您展示如何使用简历筛选器羊驼包。
您可以在 https://llamahub.ai 找到所有的羊驼包。

简历筛选器旨在根据一组标准分析候选人的简历，并决定候选人是否适合该工作。

在这个示例中，我们将评估一个样本简历（例如 Jerry 的旧简历）。


In [ ]:
%pip install llama-index-readers-wikipedia

In [ ]:
!pip install llama-index llama-hub

### 设置数据

我们将加载一些关于OpenAI、Sam、Mira和Emmett的维基百科示例数据。为什么呢？没有特别的原因 :)


In [ ]:
from llama_index.readers.wikipedia import WikipediaReader

loader = WikipediaReader()
documents = loader.load_data(
    pages=["OpenAI", "Sam Altman", "Mira Murati", "Emmett Shear"],
    auto_suggest=False,
)

In [ ]:
# 对第一段文本进行句子分割
from llama_index.core.node_parser import SentenceSplitter

In [ ]:
sentence_splitter = SentenceSplitter(chunk_size=1024)

我们从每篇文章中获取第一个块。


In [ ]:
# 为每个实体获取前1024个标记
openai_node = sentence_splitter.get_nodes_from_documents([documents[0]])[0]
sama_node = sentence_splitter.get_nodes_from_documents([documents[1]])[0]
mira_node = sentence_splitter.get_nodes_from_documents([documents[2]])[0]
emmett_node = sentence_splitter.get_nodes_from_documents([documents[3]])[0]

我们还将下载杰瑞2019年的简历。


## 从LlamaHub下载简历筛选器包

在这里，我们将从LlamaHub下载简历筛选器包类。

我们将用它来处理两种情况：
- 候选人是否适合担任前端/全栈工程师角色。
- 候选人是否适合担任OpenAI的首席执行官。


In [ ]:
from llama_index.core.llama_pack import download_llama_pack

In [ ]:
ResumeScreenerPack = download_llama_pack(
    "ResumeScreenerPack", "./resume_screener_pack"
)

### 筛选MLE角色候选人

我们从Meta的网站上获取了一份MLE角色的职位描述。


In [ ]:
meta_jd = """\
Meta 正在进行其业务和技术史上最具变革性的改变，我们的机器学习工程师处于这一演变的最前沿。通过领导关键的项目和前所未有的倡议，您有机会帮助我们推动全球人们连接方式的进步。

理想的候选人将具有在推荐、分类和优化问题领域工作的行业经验。您将带来拥有整个 ML 生命周期的能力，定义项目并在团队间推动卓越。您将与世界领先的工程师和研究人员一起解决一些存在于网络上最激动人心和庞大的社交数据和预测问题。\
"""

In [ ]:
resume_screener = ResumeScreenerPack(
    job_description=meta_jd,
    criteria=[
        "2+ years of experience in one or more of the following areas: machine learning, recommendation systems, pattern recognition, data mining, artificial intelligence, or related technical field",
        "Experience demonstrating technical leadership working with teams, owning projects, defining and setting technical direction for projects",
        "Bachelor's degree in Computer Science, Computer Engineering, relevant technical field, or equivalent practical experience.",
    ],
)

In [ ]:
response = resume_screener.run(resume_path="jerry_resume.pdf")

In [ ]:
# 遍历响应中的决策标准
for cd in response.criteria_decisions:
    print("### 决策标准")
    print(cd.reasoning)  # 输出推理
    print(cd.decision)   # 输出决策
print("#### 总体推理 ##### ")
print(str(response.overall_reasoning))  # 输出总体推理
print(str(response.overall_decision))   # 输出总体决策

### CRITERIA DECISION
Jerry Liu has more than 2 years of experience in machine learning and artificial intelligence. He worked as a Machine Learning Engineer at Quora Inc. for a year and has been an AI Research Scientist at Uber ATG since 2018. His work involves deep learning, information theory, and 3D geometry, among other areas.
True
### CRITERIA DECISION
Jerry Liu has demonstrated technical leadership in his roles at Uber ATG and Quora Inc. He has led and mentored multiple projects on multi-agent simulation, prediction, and planning. He also researched and productionized GBDT’s for new users at Quora, contributing to a 5% increase in new user active usage.
True
### CRITERIA DECISION
Jerry Liu has a Bachelor of Science in Engineering (B.S.E.) in Computer Science from Princeton University. He graduated Summa Cum Laude and was a member of Phi Beta Kappa, Tau Beta Pi, and Sigma Xi.
True
#### OVERALL REASONING ##### 
Jerry Liu meets all the screening criteria for the Machine Learning En

### 前端 / Typescript 角色的候选人筛选

在招聘前端开发或Typescript开发角色的候选人时，我们需要对候选人进行筛选，以确保他们具备所需的技能和经验。以下是一些筛选标准，可用于评估候选人是否适合前端或Typescript角色的职位。


In [ ]:
resume_screener = ResumeScreenerPack(
    job_description="We're looking to hire a front-end engineer",
    criteria=[
        "The individual needs to be experienced in front-end / React / Typescript"
    ],
)

In [ ]:
response = resume_screener.run(resume_path="jerry_resume.pdf")

In [ ]:
print(str(response.overall_reasoning))
print(str(response.overall_decision))

The candidate does not meet the specific criteria of having experience in front-end, React, or Typescript.
False


### OpenAI首席执行官候选人筛选

Jerry不会写Typescript，但他能成为OpenAI的首席执行官吗？


In [ ]:
job_description = f"""\
我们正在寻找一位OpenAI的首席执行官。

与列出一组特定标准不同，每个“标准”实际上是以前首席执行官的简短传记。

对于每个标准/传记，概述候选人的经验是否符合或超过了该标准。

另外，这里是来自维基百科关于OpenAI的描述：
{openai_node.get_content()}
"""

profile_strs = [
    f"简介：{n.get_content()}" for n in [sama_node, mira_node, emmett_node]
]


resume_screener = ResumeScreenerPack(
    job_description=job_description, criteria=profile_strs
)

In [ ]:
response = resume_screener.run(resume_path="jerry_resume.pdf")

In [ ]:
# 遍历响应中的决策标准
for cd in response.criteria_decisions:
    print("### 决策标准")
    print(cd.reasoning)  # 输出推理
    print(cd.decision)   # 输出决策
print("#### 总体推理 ##### ")
print(str(response.overall_reasoning))  # 输出总体推理
print(str(response.overall_decision))  # 输出总体决策

### CRITERIA DECISION
The candidate, Jerry Liu, has a strong background in AI research and has led multiple projects in this field. However, he does not have the same level of executive leadership experience as Samuel Harris Altman, who served as CEO of OpenAI and president of Y Combinator. Altman also has experience leading an advanced AI research team at Microsoft, which Liu does not have.
False
### CRITERIA DECISION
While Jerry Liu has a strong background in AI and machine learning, his experience does not match or surpass that of Mira Murati. Murati served as the chief technology officer of OpenAI and briefly as its interim CEO. She led the company's work on several major projects and oversaw multiple teams. Liu does not have the same level of leadership or executive experience.
False
### CRITERIA DECISION
Jerry Liu's experience does not match or surpass that of Emmett Shear. Shear co-founded Justin.tv and served as the CEO of Twitch, demonstrating significant entrepreneurial and e

抱歉，这次没有。
